In [116]:
import tensorflow as tf
from google.cloud import bigquery
import pandas as pd
import numpy as np
import warnings

# Suppres warnings
warnings.filterwarnings('ignore')

### Load Google-Analytics training data in BigQuery into a dataframe 

In [129]:
# from google.cloud import bigquery
client = bigquery.Client()

query = (
'''
SELECT 
    IF(totals.transactions IS NULL, 0, 1) AS label,
    IFNULL(device.operatingSystem, "") AS os,
    IF(device.isMobile,1,0) AS is_mobile,
    IFNULL(geoNetwork.country, "") AS country,
    IFNULL(totals.pageviews, 0) AS pageviews
FROM 
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170631'
'''
)

df = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
).to_dataframe()


In [130]:
df.head(2)

,label,os,is_mobile,country,pageviews
0,0,Windows Phone,1,India,2
1,0,Windows Phone,1,India,4


### Create and train Tensorflow Dense Neural Network model 
#### Load dataframe in dataset

In [120]:
# INPUT FUNCTION 
def pd_input_fn(df, y_label):
  return tf.estimator.inputs.pandas_input_fn(
      x=df,
      y=df[y_label],
      batch_size = 5,
      num_epochs = 5,
      shuffle = True,
      queue_capacity = 1000,
  )

#### Create the features

In [121]:
numeric_columns = ["pageviews"]
categorical_columns = ["os","country","is_mobile"]
label = "label"

feature_columns = []
for key in numeric_columns:
    feature_columns.append(tf.feature_column.numeric_column(key=key))
for key in categorical_columns:
    feature_columns.append(
        tf.feature_column.indicator_column(
            tf.feature_column.categorical_column_with_vocabulary_list(
                key=key, 
                vocabulary_list=df[key].unique())))

#### Create & train the model

In [122]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[10, 10],
    n_classes=2) # The model predicts a transaction (1) 

classifier.train(input_fn = pd_input_fn(df,label))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/ll/mvblp_pd3tsftmr2fpfx9rkm00k_rj/T/tmpo65n8z56', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x136c3f350>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done 

INFO:tensorflow:global_step/sec: 414.969
INFO:tensorflow:loss = 0.0010868596, step = 1701 (0.238 sec)
INFO:tensorflow:global_step/sec: 530.495
INFO:tensorflow:loss = 0.015475199, step = 1801 (0.187 sec)
INFO:tensorflow:global_step/sec: 545.681
INFO:tensorflow:loss = 0.0017454408, step = 1901 (0.183 sec)
INFO:tensorflow:global_step/sec: 511.347
INFO:tensorflow:loss = 0.06818641, step = 2001 (0.198 sec)
INFO:tensorflow:global_step/sec: 502.175
INFO:tensorflow:loss = 0.0017454783, step = 2101 (0.198 sec)
INFO:tensorflow:global_step/sec: 458.747
INFO:tensorflow:loss = 0.005317466, step = 2201 (0.239 sec)
INFO:tensorflow:global_step/sec: 369.45
INFO:tensorflow:loss = 0.0007738836, step = 2301 (0.250 sec)
INFO:tensorflow:global_step/sec: 570.164
INFO:tensorflow:loss = 0.001375077, step = 2401 (0.174 sec)
INFO:tensorflow:global_step/sec: 586.05
INFO:tensorflow:loss = 0.35684937, step = 2501 (0.170 sec)
INFO:tensorflow:global_step/sec: 398.073
INFO:tensorflow:loss = 0.0117913205, step = 2601 (

INFO:tensorflow:global_step/sec: 423.334
INFO:tensorflow:loss = 0.00021999356, step = 9801 (0.236 sec)
INFO:tensorflow:global_step/sec: 471.24
INFO:tensorflow:loss = 0.00031624333, step = 9901 (0.213 sec)
INFO:tensorflow:global_step/sec: 447.929
INFO:tensorflow:loss = 0.00011738872, step = 10001 (0.224 sec)
INFO:tensorflow:global_step/sec: 537.996
INFO:tensorflow:loss = 0.00035809836, step = 10101 (0.182 sec)
INFO:tensorflow:global_step/sec: 557.47
INFO:tensorflow:loss = 0.0001792236, step = 10201 (0.181 sec)
INFO:tensorflow:global_step/sec: 496.615
INFO:tensorflow:loss = 0.00034636338, step = 10301 (0.200 sec)
INFO:tensorflow:global_step/sec: 484.088
INFO:tensorflow:loss = 0.0017628786, step = 10401 (0.207 sec)
INFO:tensorflow:global_step/sec: 445.71
INFO:tensorflow:loss = 0.0003288823, step = 10501 (0.228 sec)
INFO:tensorflow:global_step/sec: 479.596
INFO:tensorflow:loss = 0.000183163, step = 10601 (0.206 sec)
INFO:tensorflow:global_step/sec: 507.701
INFO:tensorflow:loss = 0.13390289

INFO:tensorflow:loss = 0.22350998, step = 17801 (0.198 sec)
INFO:tensorflow:global_step/sec: 332.874
INFO:tensorflow:loss = 0.28085062, step = 17901 (0.300 sec)
INFO:tensorflow:global_step/sec: 497.471
INFO:tensorflow:loss = 0.0019003295, step = 18001 (0.201 sec)
INFO:tensorflow:global_step/sec: 558.683
INFO:tensorflow:loss = 0.47367454, step = 18101 (0.179 sec)
INFO:tensorflow:global_step/sec: 480.636
INFO:tensorflow:loss = 0.56017506, step = 18201 (0.207 sec)
INFO:tensorflow:global_step/sec: 579.894
INFO:tensorflow:loss = 0.0138782635, step = 18301 (0.172 sec)
INFO:tensorflow:global_step/sec: 368.801
INFO:tensorflow:loss = 0.10865487, step = 18401 (0.268 sec)
INFO:tensorflow:global_step/sec: 588.105
INFO:tensorflow:loss = 1.8594304, step = 18501 (0.172 sec)
INFO:tensorflow:global_step/sec: 383.717
INFO:tensorflow:loss = 0.030115822, step = 18601 (0.272 sec)
INFO:tensorflow:global_step/sec: 414.127
INFO:tensorflow:loss = 1.5234355, step = 18701 (0.233 sec)
INFO:tensorflow:global_step/

INFO:tensorflow:global_step/sec: 453.173
INFO:tensorflow:loss = 0.022417357, step = 25901 (0.219 sec)
INFO:tensorflow:global_step/sec: 301.439
INFO:tensorflow:loss = 0.019754255, step = 26001 (0.330 sec)
INFO:tensorflow:global_step/sec: 418.612
INFO:tensorflow:loss = 0.038889837, step = 26101 (0.243 sec)
INFO:tensorflow:global_step/sec: 431.581
INFO:tensorflow:loss = 0.06542898, step = 26201 (0.231 sec)
INFO:tensorflow:global_step/sec: 453.274
INFO:tensorflow:loss = 0.6399456, step = 26301 (0.218 sec)
INFO:tensorflow:global_step/sec: 527.184
INFO:tensorflow:loss = 0.0029415144, step = 26401 (0.190 sec)
INFO:tensorflow:global_step/sec: 365.516
INFO:tensorflow:loss = 0.00035660423, step = 26501 (0.274 sec)
INFO:tensorflow:global_step/sec: 526.412
INFO:tensorflow:loss = 0.063120134, step = 26601 (0.189 sec)
INFO:tensorflow:global_step/sec: 510.452
INFO:tensorflow:loss = 0.023214681, step = 26701 (0.197 sec)
INFO:tensorflow:global_step/sec: 482.121
INFO:tensorflow:loss = 4.615353, step = 2

...

INFO:tensorflow:loss = 0.027215607, step = 796801 (0.241 sec)
INFO:tensorflow:global_step/sec: 413.75
INFO:tensorflow:loss = 1.821647e-05, step = 796901 (0.240 sec)
INFO:tensorflow:global_step/sec: 538.773
INFO:tensorflow:loss = 5.021671e-05, step = 797001 (0.187 sec)
INFO:tensorflow:global_step/sec: 524.965
INFO:tensorflow:loss = 0.047074776, step = 797101 (0.191 sec)
INFO:tensorflow:global_step/sec: 428.705
INFO:tensorflow:loss = 0.0015919973, step = 797201 (0.233 sec)
INFO:tensorflow:global_step/sec: 456.366
INFO:tensorflow:loss = 0.00023272142, step = 797301 (0.220 sec)
INFO:tensorflow:global_step/sec: 409.264
INFO:tensorflow:loss = 2.974362e-05, step = 797401 (0.242 sec)
INFO:tensorflow:global_step/sec: 350.816
INFO:tensorflow:loss = 8.368568e-06, step = 797501 (0.289 sec)
INFO:tensorflow:global_step/sec: 299.081
INFO:tensorflow:loss = 1.9415886e-06, step = 797601 (0.329 sec)
INFO:tensorflow:global_step/sec: 261.24
INFO:tensorflow:loss = 0.00019441612, step = 797701 (0.384 sec)
IN

INFO:tensorflow:global_step/sec: 409.167
INFO:tensorflow:loss = 4.734836e-05, step = 804801 (0.244 sec)
INFO:tensorflow:global_step/sec: 446.223
INFO:tensorflow:loss = 3.133109e-05, step = 804901 (0.224 sec)
INFO:tensorflow:global_step/sec: 476.345
INFO:tensorflow:loss = 0.024931146, step = 805001 (0.209 sec)
INFO:tensorflow:global_step/sec: 529.126
INFO:tensorflow:loss = 0.002379599, step = 805101 (0.192 sec)
INFO:tensorflow:global_step/sec: 596.606
INFO:tensorflow:loss = 0.0011328215, step = 805201 (0.165 sec)
INFO:tensorflow:global_step/sec: 528.131
INFO:tensorflow:loss = 0.0024662414, step = 805301 (0.193 sec)
INFO:tensorflow:global_step/sec: 541.829
INFO:tensorflow:loss = 2.1561434e-06, step = 805401 (0.186 sec)
INFO:tensorflow:global_step/sec: 517.987
INFO:tensorflow:loss = 0.0010792253, step = 805501 (0.188 sec)
INFO:tensorflow:global_step/sec: 576.103
INFO:tensorflow:loss = 5.7692647e-05, step = 805601 (0.175 sec)
INFO:tensorflow:global_step/sec: 597.133
INFO:tensorflow:loss = 

INFO:tensorflow:global_step/sec: 397.003
INFO:tensorflow:loss = 3.8506946e-06, step = 812701 (0.250 sec)
INFO:tensorflow:global_step/sec: 470.575
INFO:tensorflow:loss = 0.00039743533, step = 812801 (0.214 sec)
INFO:tensorflow:global_step/sec: 416.525
INFO:tensorflow:loss = 0.0019886955, step = 812901 (0.238 sec)
INFO:tensorflow:global_step/sec: 491.246
INFO:tensorflow:loss = 3.3347427e-05, step = 813001 (0.207 sec)
INFO:tensorflow:global_step/sec: 471.407
INFO:tensorflow:loss = 0.0029124713, step = 813101 (0.210 sec)
INFO:tensorflow:global_step/sec: 515.349
INFO:tensorflow:loss = 0.0012577452, step = 813201 (0.193 sec)
INFO:tensorflow:global_step/sec: 416.412
INFO:tensorflow:loss = 3.6707486e-05, step = 813301 (0.259 sec)
INFO:tensorflow:global_step/sec: 387.638
INFO:tensorflow:loss = 0.15591128, step = 813401 (0.239 sec)
INFO:tensorflow:global_step/sec: 613.79
INFO:tensorflow:loss = 0.14117655, step = 813501 (0.163 sec)
INFO:tensorflow:global_step/sec: 515.844
INFO:tensorflow:loss = 3

INFO:tensorflow:global_step/sec: 342.96
INFO:tensorflow:loss = 8.468793e-05, step = 820601 (0.296 sec)
INFO:tensorflow:global_step/sec: 457.31
INFO:tensorflow:loss = 8.0877915e-05, step = 820701 (0.215 sec)
INFO:tensorflow:global_step/sec: 565.994
INFO:tensorflow:loss = 0.009087907, step = 820801 (0.176 sec)
INFO:tensorflow:global_step/sec: 337.009
INFO:tensorflow:loss = 1.0214534, step = 820901 (0.296 sec)
INFO:tensorflow:global_step/sec: 577.911
INFO:tensorflow:loss = 6.363003e-05, step = 821001 (0.176 sec)
INFO:tensorflow:global_step/sec: 488.45
INFO:tensorflow:loss = 0.059895277, step = 821101 (0.204 sec)
INFO:tensorflow:global_step/sec: 429.946
INFO:tensorflow:loss = 5.5705423e-05, step = 821201 (0.232 sec)
INFO:tensorflow:global_step/sec: 480.347
INFO:tensorflow:loss = 0.33979025, step = 821301 (0.206 sec)
INFO:tensorflow:global_step/sec: 510.952
INFO:tensorflow:loss = 0.002716929, step = 821401 (0.198 sec)
INFO:tensorflow:global_step/sec: 473.961
INFO:tensorflow:loss = 1.108819e

INFO:tensorflow:global_step/sec: 485.923
INFO:tensorflow:loss = 0.049269438, step = 828401 (0.205 sec)
INFO:tensorflow:global_step/sec: 543.777
INFO:tensorflow:loss = 3.1056268e-06, step = 828501 (0.182 sec)
INFO:tensorflow:global_step/sec: 499.715
INFO:tensorflow:loss = 0.16645704, step = 828601 (0.201 sec)
INFO:tensorflow:global_step/sec: 525.597
INFO:tensorflow:loss = 0.0001452663, step = 828701 (0.194 sec)
INFO:tensorflow:global_step/sec: 472.981
INFO:tensorflow:loss = 3.3314764e-06, step = 828801 (0.206 sec)
INFO:tensorflow:global_step/sec: 605.745
INFO:tensorflow:loss = 0.0006540614, step = 828901 (0.167 sec)
INFO:tensorflow:global_step/sec: 601.225
INFO:tensorflow:loss = 9.648288e-05, step = 829001 (0.166 sec)
INFO:tensorflow:global_step/sec: 529.317
INFO:tensorflow:loss = 2.5722088e-06, step = 829101 (0.190 sec)
INFO:tensorflow:global_step/sec: 783.773
INFO:tensorflow:loss = 0.00019706062, step = 829201 (0.124 sec)
INFO:tensorflow:Saving checkpoints for 829285 into /var/folders

In [123]:
predictions_query = (
'''
SELECT 
    IF(totals.transactions IS NULL, 0, 1) AS label,
    IFNULL(device.operatingSystem, "") AS os,
    IF(device.isMobile,1,0) AS is_mobile,
    IFNULL(geoNetwork.country, "") AS country,
    IFNULL(totals.pageviews, 0) AS pageviews
FROM 
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'
LIMIT
    10
'''
)

df_predict = client.query(
    predictions_query,
    location="US",
).to_dataframe()

In [124]:
df_predict.head(5)

,label,os,is_mobile,country,pageviews
0,0,Windows,0,Canada,4
1,0,Windows,0,Canada,4
2,0,Windows,0,India,5
3,0,Windows,0,Brazil,12
4,0,Android,1,India,3


In [125]:
predictions = classifier.predict(input_fn = pd_input_fn(df_predict,label))
for prediction in predictions:
    print("Predicted class {} probabilities {}".format(prediction['class_ids'], prediction['probabilities']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/ll/mvblp_pd3tsftmr2fpfx9rkm00k_rj/T/tmpo65n8z56/model.ckpt-829285
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predicted class [0] probabilities [9.9999952e-01 4.6401686e-07]
Predicted class [0] probabilities [9.9999940e-01 5.8657895e-07]
Predicted class [0] probabilities [9.9939024e-01 6.0979417e-04]
Predicted class [0] probabilities [9.999354e-01 6.460896e-05]
Predicted class [0] probabilities [9.9998808e-01 1.1874816e-05]
Predicted class [0] probabilities [9.9999976e-01 2.4357831e-07]
Predicted class [0] probabilities [9.999354e-01 6.460896e-05]
Predicted class [0] probabilities [9.9999762e-01 2.4143499e-06]
Predicted class [0] probabilities [9.999354e-01 6.460896e-05]
Predicted class [0] probabilities [9.9999762e-01 2.4143496e-06]
Predicted class [0] probabilities [9.9998808e-01 1.1

In [126]:
# SERVING FUNCTION, SPECIFY WHAT DATA HAS TO LOOK LIKE AT SERVE TIME
def serving_input_fn():
    feature_placeholders = {
        'is_mobile' : tf.placeholder(tf.int16,[None]),
        'os' : tf.placeholder(tf.string,[None]),
        'country' : tf.placeholder(tf.string,[None]),
        'pageviews' : tf.placeholder(tf.int16,[None])
    }
        
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    return tf.estimator.export.ServingInputReceiver(features, 
                                                    feature_placeholders)

In [127]:
# EXPORT MODEL
gcs_path = "gs://rabobqml/model_output" 
classifier.export_savedmodel(gcs_path,serving_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'serving_default' : Classification input must be a single string Tensor; got {'is_mobile': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int16>, 'os': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=string>, 'country': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=string>, 'pageviews': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=int16>}
INFO:tensorflow:'classification' : Classification input must be a single string Tensor; got {'is_mobile': <tf.Tensor 'Placeholder:0' shape=(?

b'gs://rabobqml/model_output/1568364406'